In [1]:
import json
import os
from pathlib import Path

import histalign.backend.io as io
import matplotlib.pyplot as plt
import numpy as np
import scipy.interpolate as interpolate
import scipy.ndimage as ndimage
import vedo

from histalign.backend.ccf.allen_downloads import get_structure_path
from histalign.backend.models import AlignmentParameterAggregator, VolumeSettings
from histalign.backend.registration.ReverseRegistrator import ReverseRegistrator
from histalign.backend.workspace import VolumeManager

ModuleNotFoundError: No module named 'histalign.backend.ccf.allen_downloads'

In [ ]:
vedo.settings.default_backend = "vtk"

In [3]:
def get_parameters(alignment_path: str) -> AlignmentParameterAggregator:
    with open(alignment_path) as handle:
        return AlignmentParameterAggregator(**json.load(handle))


def mask_off(
    image_array: np.ndarray,
    structure_name: str,
    parameters: AlignmentParameterAggregator,
) -> np.ndarray:
    mask_volume = VolumeManager()
    mask_volume.load_volume(get_structure_path(structure_name, parameters.resolution))

    volume_settings = VolumeSettings()
    volume_settings.offset = parameters.offset

    mask_slice = mask_volume.slice_volume(volume_settings)

    reverse_registrator = ReverseRegistrator(True, True)
    registered_image_array = reverse_registrator.get_forwarded_image(
        parameters, image_array
    )

    return np.where(mask_slice > 0, registered_image_array, 0)


def insert_into_array(
    minor_array: np.ndarray,
    master_array: np.ndarray,
    parameters: AlignmentParameterAggregator,
) -> np.ndarray:
    def convert_offset_to_index(offset: int, resolution: int) -> int:
        anti_offset_map = {100: 57, 50: 114, 25: 228, 10: 270}
        return offset + anti_offset_map[resolution]

    index = convert_offset_to_index(parameters.offset, parameters.resolution)
    master_array[..., index] = minor_array
    return master_array


def process_alignment(
    alignment_path: str,
    structure_name: str,
    master_array: np.ndarray,
    neun: bool = True,
) -> np.ndarray:
    print("Getting parameters")
    parameters = get_parameters(str(alignment_path))
    print("Getting raw image")
    if neun:
        raw_image = io.load_image(
            parameters.histology_file_path, normalise_dtype=np.uint8
        )
    else:
        path = parameters.histology_file_path.replace("channel1", "channel2").replace(
            "470_New 1", "700_New 2"
        )
        raw_image = io.load_image(path, normalise_dtype=np.uint8)
    print("Getting masked image")
    masked_image = mask_off(raw_image, structure_name, parameters).T

    return insert_into_array(masked_image, master_array, parameters)


def imshow(image: np.ndarray) -> None:
    _, axes = plt.subplots()

    axes.imshow(image)
    axes.axis(False)

    plt.show()

In [4]:
#
results_directory = "/home/ediun/git/histalign/projects/project_310724_full/7d9af388b0"

project_file = str(Path(results_directory).parent / "project.json")

with open(project_file) as handle:
    project_resolution = json.load(handle)["atlas_resolution"]

#
alignment_paths = []
for path in Path(results_directory).iterdir():
    if path.is_dir() or path.stem == "metadata" or path.suffix != ".json":
        continue

    alignment_paths.append(path)

#
neun = False
volume_array_path = (
    f"notebooks/resources/volume_array_{'neun' if neun else 'mecp2'}.npy"
)

if os.path.exists(volume_array_path):
    volume_array = np.load(volume_array_path)
else:
    dimensions_map = {
        100: (132, 80, 114),
        50: (264, 160, 228),
        25: (528, 320, 456),
        10: (1320, 800, 1140),
    }

    dimensions = dimensions_map[project_resolution]

    volume_array = np.zeros(dimensions)

    i = 0
    for alignment_path in alignment_paths:
        i += 1
        print(f"Processing {i}")

        volume_array = process_alignment(
            alignment_path, "Isocortex", volume_array, neun
        )

    # Ensuring we have only unique offsets
    j = 0
    for k in range(volume_array.shape[-1]):
        if volume_array[..., k].max() > 0:
            j += 1

    assert i == j, "Some offsets were the same, previous slices were overwritten"

    np.save(volume_array_path, volume_array)

In [5]:
mask_manager = VolumeManager()
mask_manager.load_volume(get_structure_path("Isocortex", project_resolution))
mask: vedo.Volume = mask_manager._volume

In [6]:
volume_array = volume_array
mask_array = mask.tonumpy()

# Smoothing/interpolating

In [7]:
blurred = ndimage.gaussian_filter(volume_array, sigma=8, radius=64)

In [8]:
masked_blurred = np.where(mask_array > 0, blurred, 0)

In [ ]:
volume = vedo.Volume(masked_blurred).tomesh()
volume.opacity(0.5)

volume.show()